In [11]:
######## SPAIN - SHP FILES

import geopandas as gpd

# Check the georeference stystem
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Spain/Portbou/Portbou.shp')
print(gdf.crs)

gdf = gdf.to_crs(epsg=32719)
print(gdf.crs)


EPSG:4326
EPSG:32719


In [ ]:
###### LA ESTRELLA 2020
import pandas as pd
import geopandas as gpd
import re

gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Spain/La_Estrella_2020/Isochrones/Isochrones.shp')
gdf = gdf.to_crs(epsg=32719)

def crear_iso_timestamp(row):
    # Extract date from 'fecha' and format 30072020 → 2020-07-30
    date_str = str(row['fecha'])
    if len(date_str) == 8:
        date_fmt = f"{date_str[4:8]}-{date_str[2:4]}-{date_str[0:2]}"
    else:
        date_fmt = ''
    # Extract hour from 'Descripcio'
    descripcio_str = str(row['Descripcio'])
    m = re.search(r'_(\d{4})$', descripcio_str)
    if m:
        hour_fmt = f"{m.group(1)[0:2]}:{m.group(1)[2:4]}"
    else:
        hour_fmt = ''
    # COmbine date and hour
    if date_fmt and hour_fmt:
        return f"{date_fmt} {hour_fmt}"
    else:
        return ''

# Create the new column
gdf['iso_timestamp'] = gdf.apply(crear_iso_timestamp, axis=1)

# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Delete attribute
gdf = gdf.drop('Perim_m', axis=1)
gdf = gdf.drop('Superficie', axis=1)
gdf = gdf.drop('Id', axis=1)

gdf = gdf.sort_values('iso_timestamp')

gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2.to_file('LaEstrella2020Fire_test1.geojson', driver='GeoJSON')


In [6]:
###### LA ESTRELLA 2024
import pandas as pd
import geopandas as gpd


gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Spain/La_Estrella_2024/Isochrones/Isochrones.shp')
gdf = gdf.to_crs(epsg=32719)

# COnvert 'Fecha_Hora' to 'iso_timestamp'
gdf['iso_timestamp'] = pd.to_datetime(
    gdf['Fecha_Hora'], 
    format='%Y%m%d_%H%M',   # Indicate original format
    errors='coerce'          
).dt.strftime('%Y-%m-%d %H:%M')  # Indicate format we want


# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Delete attribute
gdf = gdf.drop('Sup_Ha', axis=1)
gdf = gdf.drop('ORIG_FID', axis=1)
gdf = gdf.drop('Fecha_Hora', axis=1)
gdf = gdf.drop('Dissolve', axis=1)
gdf = gdf.drop('Perimetro', axis=1)

gdf = gdf.sort_values('iso_timestamp')

gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2.to_file('LaEstrella2024Fire_test1.geojson', driver='GeoJSON')

In [9]:
##### PONT VILOMARA
import geopandas as gpd

# Check the georeference stystem
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Spain/Pont_de_Vilomara/Pont_Vilomara.shp')
gdf = gdf.to_crs(epsg=32719)

# Rename attributes
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('FID', axis=1)
gdf = gdf.drop('zp_link', axis=1)
gdf = gdf.drop('burn_perio', axis=1)

# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

gdf = gdf.sort_values('iso_timestamp')

gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2.to_file('PontVilomaraFire_test1.geojson', driver='GeoJSON')


In [15]:
##### PORTBOU
import geopandas as gpd

# Check the georeference stystem
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Spain/Portbou/Portbou.shp')
gdf = gdf.to_crs(epsg=32719)

# Rename attributes
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Delete ALL attributes except some
attrib_keep = ['iso_timestamp', 'type', 'source']
gdf = gdf[attrib_keep + ['geometry']]

# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

gdf = gdf.sort_values('iso_timestamp')

gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2.to_file('PortbouFire_test1.geojson', driver='GeoJSON')

In [4]:
####### SANTA CRUZ MUDELA 
import geopandas as gpd
import pandas as pd
import os
import re

# First convine all the shapefiles in one
folder = '/home/polivera/Documents/NERO - Database/Spain/Santa_Cruz_de Mudela_2015/Isochrones'
files = [f for f in os.listdir(folder) if f.lower().endswith('.shp')]

gdf_list = []

for file in files:
    name = file.replace('.shp','')

    # Date: firts 8 digits
    date = name[:8]
    # Hour: last 4 digits
    hour_match = re.search(r'_(\d{4})$', name)
    hour = hour_match.group(1) if hour_match else ''
    
    if len(date)==8 and len(hour)==4:
        iso_timestamp = f"{date[:4]}-{date[4:6]}-{date[6:8]} {hour[:2]}:{hour[2:]}"
    else:
        iso_timestamp = ''
    
    gdf = gpd.read_file(os.path.join(folder, file))
    # Add 'iso_timestamp' attribute 
    gdf['iso_timestamp'] = iso_timestamp
    gdf_list.append(gdf)

    # Make only one shapefile
gdf_conv = pd.concat(gdf_list, ignore_index=True)
gdf_conv = gpd.GeoDataFrame(gdf_conv, geometry='geometry')

### Here edit the new shapefile
# Delete attributes
gdf_conv = gdf_conv.drop('area_ha', axis=1, errors='ignore')
gdf_conv = gdf_conv.drop('peri_m', axis=1, errors='ignore')
gdf_conv = gdf_conv.drop('Id', axis=1, errors='ignore')

# Add new attributes
gdf_conv['area_ha'] = gdf_conv.geometry.area / 10000  
gdf_conv['perimeter_m'] = gdf_conv.geometry.length

# Sort attributes
gdf_conv = gdf_conv.sort_values('iso_timestamp')

gdf_2 = gdf_conv.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2.to_file('StaCruzMudelaFire_test2.geojson', driver='GeoJSON')
